<a href="https://colab.research.google.com/github/vannguyenhoai43-hash/ecommerce-category-growth-analysis/blob/main/notebooks/03_growth_driver_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Phần 3: Phân tích động lực tăng trưởng

Mục tiêu của phần này là xác định những danh mục Level 2 và nhóm sản phẩm chính đang đóng góp vào sự thay đổi tăng/giảm của toàn ngành theo tháng.

**Bước 1: Xác định danh mục Level 2 đóng góp chính**

- Tính mức tăng trưởng tuyệt đối (MoM) của từng Level 2

- Xác định:

  - Top 3 Level 2 tăng mạnh nhất

  - Top 3 Level 2 suy giảm nhiều nhất

- Tính tỷ trọng đóng góp của từng Level 2 vào tổng mức tăng/giảm toàn ngành

Mục tiêu: xác định đâu là top tăng trưởng và đâu là nhóm đang kéo giảm hiệu suất chung.

**Bước 2: Phân tích sâu theo nhóm sản phẩm nổi bật**

Sau khi xác định các Level 2 ở trên , tiến hành phân tích sâu ở cấp độ sản phẩm (đã được chuẩn hóa và nhóm theo keywords):

- Đối với Level 2 tăng trưởng mạnh:
  - Top down các sản phẩm có mức diff cao nhất, lấy top 3 sản phẩm.

- Đối với Level 2 suy giảm hoặc tăng trưởng chậm:

  - Tập trung vào các sản phẩm ADO/GMV tháng trước cao nhưng giảm mạnh ở tháng này: lấy top 20 sản phẩm có mức ADO/GMV cao nhất ở tháng trước, rồi lấy top 3 sản phẩm từ thấp tới cao.

Mục tiêu: làm rõ tăng trưởng/ suy giảm đến từ nhóm sản phẩm nào.

## Chuẩn bị data

### Import data:

In [52]:
%cd /content
!rm -rf ecommerce-category-growth-analysis

/content


In [53]:
!git clone https://github.com/vannguyenhoai43-hash/ecommerce-category-growth-analysis.git

Cloning into 'ecommerce-category-growth-analysis'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (124/124), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 124 (delta 46), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (124/124), 1.15 MiB | 4.50 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
import sys
sys.path.append("/content/ecommerce-category-growth-analysis/src")

from load_data import load_data

In [6]:
# DF theo level 2 với cur_month = 20024-09
df_cur_lv2= load_data(month='2024-09')['cur_lv2']

In [7]:
# DF theo items với cur_month = 20024-09
df_cur_items= load_data(month='2024-09')['cur_items']

## Triển khai

In [9]:
# Tăng trưởng theo level 1 + level 2
from metrics import growth_by_mul_level
l1_L2_growth = growth_by_mul_level(df_cur_lv2, ['level2_kpi_category', 'level1_kpi_category'])
l1_L2_growth_sort = l1_L2_growth.sort_values('level1_kpi_category')
print(l1_L2_growth_sort)

      diff_ado     diff_gmv  grow_ado  grow_gmv         ADO_M       ADO_M_1  \
8   290.833185  2088.187566  0.280875  0.335255   1326.289140   1035.455955   
1   240.151542  1166.658283  0.236101  0.182195   1257.305894   1017.154352   
2    80.495411  -606.060025  0.044013 -0.059055   1909.389789   1828.894378   
3   -92.698572 -1305.599699 -0.115163 -0.219878    712.234388    804.932960   
4    56.939348 -1074.727565  0.036420 -0.138811   1620.329208   1563.389860   
5     9.610908  5077.185056  0.074821  0.420203    138.062586    128.451678   
14 -278.323065 -3071.923313 -0.038126 -0.124010   7021.828669   7300.151734   
13    0.018279  -107.672716  0.033332 -0.130589      0.566667      0.548388   
15 -217.429876 -2871.259945 -0.017937 -0.042728  11904.668096  12122.097972   
10  410.234619  1375.558008  0.126093  0.079120   3663.657544   3253.422925   
11  359.185292  3088.709561  0.207875  0.196891   2087.075546   1727.890254   
12 -159.927383  1376.460112 -0.039813  0.041144   38

In [10]:
# Chuyển item_name về các nhóm keywords
from metrics import clean_and_shorten
item_df = df_cur_items.copy()
item_df ['keywords']= item_df ['item_name'].apply(clean_and_shorten)

## Top 3 ngành hàng có mức tăng trưởng cao nhất

In [12]:
# Tính % đóng góp tăng trưởng
df_contrib= l1_L2_growth_sort.copy()
# Theo ADO
df_contrib['total_ado_growth'] = df_contrib.groupby('level1_kpi_category')['diff_ado'].transform('sum')
df_contrib['contrib_ado'] = df_contrib['diff_ado']/df_contrib['total_ado_growth']

In [42]:
# Top ngành hàng level 2 có mức tăng trưởng tốt (top 3 diff_ado>0)
highest_ado_contrib = (df_contrib
                  .query("diff_ado > 0")
                  .sort_values(by=['level1_kpi_category','diff_ado'],ascending=[True,False])
                  .groupby('level1_kpi_category')
                  .head(3)
                  .assign(contrib_ado =lambda x: (x['contrib_ado']*100).round(2).astype(str)+"%")
                  [['level1_kpi_category','level2_kpi_category', 'diff_ado','contrib_ado']]
)
highest_ado_contrib

,level1_kpi_category,level2_kpi_category,diff_ado,contrib_ado
10,Automotive,Helmets,410.234619,47.76%
11,Automotive,Interior Accessories,359.185292,41.81%
8,Automotive,Exterior Accessories,290.833185,33.86%
0,Tools & Home Improvement,Accessories,500.957482,87.86%
6,Tools & Home Improvement,Building and construction,290.342090,50.92%


### Top sản phẩm tăng trưởng

In [23]:
# tăng trưởng theo level 1 + level 2 + items
item_growth = growth_by_mul_level(item_df, ['level1_kpi_category','level2_kpi_category','keywords'])
item_growth_sort = item_growth.sort_values('level1_kpi_category')
print(item_growth_sort)

        diff_ado   diff_gmv  grow_ado  grow_gmv      ADO_M   ADO_M_1  \
0      51.163095  93.716135  0.000000  0.000000  51.163095  0.000000   
26790   0.050538   0.807812  3.133362  1.636360   0.066667  0.016129   
26791   0.005556   0.097650  0.000000  0.000000   0.005556  0.000000   
26792   0.105897   1.790232  0.537186  0.767935   0.303030  0.197133   
26793   0.033333   0.183533  0.000000  0.000000   0.033333  0.000000   
...          ...        ...       ...       ...        ...       ...   
50618   0.001075  -0.734667  0.033325 -0.655667   0.033333  0.032258   
50617   0.005574   1.024734  0.019259  0.349086   0.295000  0.289426   
50616   0.324283   1.189397  0.000000  0.000000   0.324283  0.000000   
50614   0.016667   0.281448  0.000000  0.000000   0.016667  0.000000   
71495   0.070370   0.450469  0.000000  0.000000   0.070370  0.000000   

         AdGMV_M  AdGMV_M_1       level1_kpi_category  \
0      93.716135   0.000000                Automotive   
26790   1.301476   0.

In [24]:
#Lọc top ado level 2
level2_top_ado_list = (
    highest_ado_contrib
    .loc[highest_ado_contrib['level2_kpi_category'] != 'Others',
         'level2_kpi_category']
    .unique()
    .tolist()
)

level2_top_ado_list

['Helmets',
 'Interior Accessories',
 'Exterior Accessories',
 'Accessories',
 'Building and construction',
 'Handtool']

In [25]:
#Danh sách các ngành hàng là top level 2
df_ado_l1_l2_item = item_growth[
    item_growth['level2_kpi_category'].isin(level2_top_ado_list)
].copy()

In [35]:
# Danh sách top 3 sản phẩm theo cat có mức tăng trưởng tốt
from metrics import top3_items
top3_items_ado =  top3_items(item_growth,'ado')
top3_items_ado_sort = top3_items_ado.sort_values(['level1_kpi_category','level2_kpi_category'],ascending = [True,True])
top3_items_ado_sort

,level1_kpi_category,level2_kpi_category,keywords,diff_ado
41,Automotive,Auto Parts & Spares,10 vít vá lốp xe,85.898357
0,Automotive,Auto Parts & Spares,01 dây đèn led mí,51.163095
1597,Automotive,Auto Parts & Spares,đèn led nháy audi lùi,25.840873
2053,Automotive,Automotive Care,bút xoá vết trầy xước,122.741957
3112,Automotive,Automotive Care,khử mùi thất xe ô,27.793365
3314,Automotive,Automotive Care,nano kính chống bám thương,14.889534
4828,Automotive,Automotive Oils & Lubes,hòa dyramid xăng con vệ,33.725391
5209,Automotive,Automotive Oils & Lubes,nhớt xe máy tay ga,17.023387
4613,Automotive,Automotive Oils & Lubes,dưỡng sên phục hồi nhữa,14.915628
5569,Automotive,Bicycle & E-bike Accessories,bơm xe đạp áp suất,34.068636


##Top 3 ngành hàng có mức giảm cao nhất

In [59]:
df_lowest_l2_ado = (l1_L2_growth
                  .query("diff_ado < 0")
                  .sort_values(by=['level1_kpi_category','diff_ado'],ascending=[True,True])
                  .groupby('level1_kpi_category')
                  .head(3)
                  [['level1_kpi_category','level2_kpi_category', 'diff_ado']]
)
df_lowest_l2_ado

,level1_kpi_category,level2_kpi_category,diff_ado
14,Automotive,Motorbike Accessories,-278.323065
15,Automotive,Motorbike Spare Parts,-217.429876
3,Automotive,Automotive Oils & Lubes,-92.698572
12,Tools & Home Improvement,Large tools and equipment,-159.927383
16,Tools & Home Improvement,Safety and security,-25.699967
7,Tools & Home Improvement,Electrical Circuitry & Parts,-24.213226


In [60]:
# Danh sách ngành hàng level 2 mức giảm nhiều nhất
low_grow_ado_cat = df_lowest_l2_ado['level2_kpi_category'].unique()
low_grow_ado_cat

array(['Motorbike Accessories', 'Motorbike Spare Parts',
       'Automotive Oils & Lubes', 'Large tools and equipment',
       'Safety and security', 'Electrical Circuitry & Parts'],
      dtype=object)

In [68]:
# Danh sách sản phẩm theo cat có mức giảm nhiều nhất
df_low_ado_items = item_growth[
    item_growth['level2_kpi_category'].isin(low_grow_ado_cat)
].copy()

In [70]:
# TÍnh tổng level 2
df_contrib_low_ado_items = df_low_ado_items.copy()
# Theo ADO
df_contrib_low_ado_items['total_ado_grow'] = df_contrib_low_ado_items.groupby('level2_kpi_category')['diff_ado'].transform('sum')
df_contrib_low_ado_items['contrib_ado'] = df_contrib_low_ado_items['diff_ado']/df_contrib_low_ado_items['total_ado_grow']

In [71]:
# Danh sách top 3 sản phẩm theo cat có mức giảm nhiều nhất (top 20 ADO_M_1 và top 3 diff_ADO giảm nhiều nhất)
from metrics import top3_items_low
top3_items_low_ado =  top3_items_low(df_contrib_low_ado_items,'ado')
top3_items_low_ado_sort = top3_items_low_ado.sort_values(['level1_kpi_category','level2_kpi_category'],ascending = True)
top3_items_low_ado_sort

,level1_kpi_category,level2_kpi_category,keywords,ADO_M,ADO_M_1,diff_ado,contrib_ado
4952,Automotive,Automotive Oils & Lubes,mỡ tra bảo dưỡng xe,16.416339,28.637941,-12.221602,-0.074411
5359,Automotive,Automotive Oils & Lubes,vệ sinh sên xích bảo,6.184444,9.636380,-3.451936,-0.021017
4913,Automotive,Automotive Oils & Lubes,mỡ bò bảo dưỡng sên,1.434444,4.373656,-2.939212,-0.017895
17918,Automotive,Motorbike Accessories,gương 5 cạnh rizoma nhôm,76.257434,120.226839,-43.969405,-0.024945
17512,Automotive,Motorbike Accessories,dè fi sirius 50 110,21.193889,40.215054,-19.021165,-0.010791
18090,Automotive,Motorbike Accessories,gương cờ việt nam gắn,4.113334,21.751612,-17.638278,-0.010007
27478,Automotive,Motorbike Spare Parts,cùm tăng tốc domino bao,1.435834,37.627857,-36.192023,-0.012934
38162,Automotive,Motorbike Spare Parts,đèn nháy audi stop cực,29.940782,63.229518,-33.288736,-0.011896
29362,Automotive,Motorbike Spare Parts,gác chân đúc gcraft long,32.656415,56.278354,-23.621939,-0.008442
47968,Tools & Home Improvement,Electrical Circuitry & Parts,công tắc điều khiển từ,123.549212,147.427763,-23.878551,-0.034351
